In [1]:
import os,sys
import pandas as pd
import numpy as np

In [ ]:
os.chdir("/projects/cc/kxsw653/ExCAPE-ML_v5/IT4I-data/qsar_models/")

# Functions

In [3]:
# HP SELECTION - INNER LOOP
# find out the best HP per fold
def selectHP(metric, perf_inner):
    data1 = []
    data2 = []
    data3 = []
    for target in perf_inner['target'].unique():
        # extract results for the current target
        sub = perf_inner.loc[perf_inner['target']==target]
    
        # get AUC performance averaged for each given fold i.e. averaged over the two inner folds
        subgrouped3 = sub.groupby(['ofold', 'hp']).mean().reset_index().sort_values('auc', ascending=False)    
        
        
        # extract best AUC for each of the three folds
        sub1 = subgrouped3.loc[subgrouped3['ofold']==1].sort_values(metric, ascending=False).drop(['ofold','ifold','auc','recall','precision','kappa'],axis=1).head(1)
        sub2 = subgrouped3.loc[subgrouped3['ofold']==2].sort_values(metric, ascending=False).drop(['ofold','ifold','auc','recall','precision','kappa'],axis=1).head(1)
        sub3 = subgrouped3.loc[subgrouped3['ofold']==3].sort_values(metric, ascending=False).drop(['ofold','ifold','auc','recall','precision','kappa'],axis=1).head(1)

        data1.append(sub1.values[0])
        data2.append(sub2.values[0])
        data3.append(sub3.values[0])
        
    hp_fold1 = pd.DataFrame(data1, columns=[ 'hp_fold1','target'])
    hp_fold2 = pd.DataFrame(data2, columns=[ 'hp_fold2','target'])
    hp_fold3 = pd.DataFrame(data3, columns=[ 'hp_fold3','target'])
    
    return hp_fold1.merge(hp_fold2, on='target').merge(hp_fold3, on='target')

In [4]:
# Find out the performance - OUTER LOOP
# using the HP selected in inner loop
def getOuterLoopPerformance(perf_outer, hp_select):    
    
    data1,data2,data3=[],[],[]
    for i in range(526):
        
        hp_fold1 = hp_selection.loc[hp_selection['target']==i]['hp_fold1'].values[0]
        hp_fold2 = hp_selection.loc[hp_selection['target']==i]['hp_fold2'].values[0]
        hp_fold3 = hp_selection.loc[hp_selection['target']==i]['hp_fold3'].values[0]
        
        perf_fold1 = perf_outer.loc[(perf_outer['target']==i)&(perf_outer['hp']==hp_fold1)&(perf_outer['ofold']==1)]
        perf_fold2 = perf_outer.loc[(perf_outer['target']==i)&(perf_outer['hp']==hp_fold2)&(perf_outer['ofold']==2)]
        perf_fold3 = perf_outer.loc[(perf_outer['target']==i)&(perf_outer['hp']==hp_fold3)&(perf_outer['ofold']==3)]

        ####  one target missing for SVM and RF
        if perf_fold1.shape[0]==0 or perf_fold2.shape[0]==0 or perf_fold3.shape[0]==0:continue
        ####
                
        data1.append(perf_fold1.values[0])
        data2.append(perf_fold2.values[0])
        data3.append(perf_fold3.values[0])
        
        
    all_perf_fold1 = pd.DataFrame(data1, columns=perf_fold1.columns.values)
    all_perf_fold2 = pd.DataFrame(data2, columns=perf_fold2.columns.values)
    all_perf_fold3 = pd.DataFrame(data3, columns=perf_fold3.columns.values)
    
    return all_perf_fold1,all_perf_fold2,all_perf_fold3

# XGboost

## Inner fold - HP search

In [9]:
xgboost_df = pd.read_csv('results_v5_xgboost_ecfp/all-nested-cv-perf-scores.csv', header=0, sep="\t")

# separate inner and out loop results
xgb_inner = xgboost_df.loc[xgboost_df['ifold'].isin([1,2])].copy(deep=True).drop(['al','sample'],axis=1)
xgb_outer = xgboost_df.loc[xgboost_df['ifold'].isin([0])].copy(deep=True).drop(['al','sample'],axis=1)
del xgboost_df

In [7]:
xgb_inner.head()

,ofold,ifold,target,hp,auc,recall,precision,kappa
9468,1,1,0,booster_gbtree-learning_rate_0.05-max_depth_5-...,0.808374,0.702326,0.731826,0.507079
9469,1,1,0,booster_gbtree-learning_rate_0.05-max_depth_5-...,0.804593,0.857364,0.603053,0.404244
9470,1,1,0,booster_gbtree-learning_rate_0.05-max_depth_5-...,0.792723,0.913178,0.523556,0.253211
9471,1,1,0,booster_gbtree-learning_rate_0.05-max_depth_5-...,0.810966,0.699225,0.722756,0.495144
9472,1,1,0,booster_gbtree-learning_rate_0.05-max_depth_5-...,0.810930,0.865116,0.605863,0.412486


In [8]:
# select best HP per fold
hp_selection = selectHP('auc', xgb_inner)

In [14]:
hp_selection.head()

,hp_fold1,target,hp_fold2,hp_fold3
0,booster_gbtree-learning_rate_0.05-max_depth_10...,0.0,booster_gbtree-learning_rate_0.05-max_depth_10...,booster_gbtree-learning_rate_0.05-max_depth_10...
1,booster_gbtree-learning_rate_0.05-max_depth_5-...,100.0,booster_gbtree-learning_rate_0.05-max_depth_5-...,booster_gbtree-learning_rate_0.05-max_depth_10...
2,booster_gbtree-learning_rate_0.05-max_depth_5-...,101.0,booster_gbtree-learning_rate_0.05-max_depth_10...,booster_gbtree-learning_rate_0.05-max_depth_5-...
3,booster_gbtree-learning_rate_0.05-max_depth_5-...,102.0,booster_gbtree-learning_rate_0.05-max_depth_5-...,booster_gbtree-learning_rate_0.05-max_depth_5-...
4,booster_gbtree-learning_rate_0.05-max_depth_10...,103.0,booster_gbtree-learning_rate_0.05-max_depth_5-...,booster_gbtree-learning_rate_0.05-max_depth_10...


## Outer folds - Models evaluations

In [15]:
# use the selected HP to find out the performance in outer loop
perf_fold1, perf_fold2, perf_fold3 = getOuterLoopPerformance(xgb_outer, hp_selection)
perf_fold1.to_csv("results_v5_xgboost_ecfp/perf_fold1.csv", sep="\t", index=None)
perf_fold2.to_csv("results_v5_xgboost_ecfp/perf_fold2.csv", sep="\t", index=None)
perf_fold3.to_csv("results_v5_xgboost_ecfp/perf_fold3.csv", sep="\t", index=None)

In [9]:
# hp selection based on AUC
print("           fold1           fold2           fold3")
print("AUC      =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['auc'].mean(),
                                                                           perf_fold1['auc'].std(),
                                                                           perf_fold2['auc'].mean(),
                                                                           perf_fold2['auc'].std(),
                                                                           perf_fold3['auc'].mean(),
                                                                           perf_fold3['auc'].std()))

print("PRECISION=({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['precision'].mean(),
                                                                           perf_fold1['precision'].std(),
                                                                           perf_fold2['precision'].mean(),
                                                                           perf_fold2['precision'].std(),
                                                                           perf_fold3['precision'].mean(),
                                                                           perf_fold3['precision'].std()))

print("RECALL   =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['recall'].mean(),
                                                                           perf_fold1['recall'].std(),
                                                                           perf_fold2['recall'].mean(),
                                                                           perf_fold2['recall'].std(),
                                                                           perf_fold3['recall'].mean(),
                                                                           perf_fold3['recall'].std()))

print("KAPPA    =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['kappa'].mean(),
                                                                           perf_fold1['kappa'].std(),
                                                                           perf_fold2['kappa'].mean(),
                                                                           perf_fold2['kappa'].std(),
                                                                           perf_fold3['kappa'].mean(),
                                                                           perf_fold3['kappa'].std()))

           fold1           fold2           fold3
AUC      =(0.8084+/-0.1212,0.8067+/-0.1224,0.8106+/-0.1177)
PRECISION=(0.6579+/-0.2562,0.6549+/-0.2602,0.6677+/-0.2410)
RECALL   =(0.5599+/-0.3462,0.5524+/-0.3426,0.5577+/-0.3394)
KAPPA    =(0.3626+/-0.2411,0.3619+/-0.2339,0.3649+/-0.2359)


## find out best HP out of three outer folds

In [11]:
# get the best HP out of the three HP selected in inner loop based on outer loop performances
best_hp_for_full_scale = []
for i in range(526):
    
    best_fold = np.argmax(np.array([perf_fold1.iloc[i]['auc'],
                                    perf_fold2.iloc[i]['auc'],
                                    perf_fold3.iloc[i]['auc']]))
    
    
    best_hp = np.array([perf_fold1.iloc[i]['hp'],
                        perf_fold2.iloc[i]['hp'],
                        perf_fold3.iloc[i]['hp']])[best_fold]
    
    
    best_hp_for_full_scale.append(best_hp)


In [ ]:
# select one best HP per target based on full outer loop perf
# 

In [39]:
best_hp_for_full_scale = xgb_outer.sort_values('auc',ascending=False).groupby(['target']).first().reset_index()['hp'].values

In [40]:
# generate commands for full-scale models to run 
#
# Usage: python xgboost_model.py [target_idx] [obj] [lr] [scale_pos_w] [n_Estim] [max_depth]
srun_cmd = 'srun --partition=core -c 12 --time=3-00:00:00 -J xgb'

for i in range(526):
    target  = i
    hp      = best_hp_for_full_scale[i].split('-')
    booster = hp[0].replace('booster_', '')
    lr      = hp[1].replace('learning_rate_', '')
    max_d   = hp[2].replace('max_depth_', '')
    n_estim = hp[3].replace('n_estimators_', '')
    obj     = hp[4].replace('objective_', '')
    scale_pw= hp[5].replace('scale_pos_weight_', '')
    print("{}{} python xgboost_model.py {} {} {} {} {} {} & ".format(srun_cmd, i, target, obj, lr, scale_pw, n_estim, max_d))

    
    

srun --partition=core -c 12 --time=3-00:00:00 -J xgb0 python xgboost_model.py 0 binary:logistic 0.05 1 200 10 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb1 python xgboost_model.py 1 binary:logistic 0.05 1 200 5 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb2 python xgboost_model.py 2 binary:logistic 0.05 1 200 5 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb3 python xgboost_model.py 3 binary:logistic 0.05 10 200 10 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb4 python xgboost_model.py 4 binary:logistic 0.05 1 200 10 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb5 python xgboost_model.py 5 binary:logistic 0.05 5 200 5 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb6 python xgboost_model.py 6 binary:logistic 0.05 1 200 10 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb7 python xgboost_model.py 7 binary:logistic 0.05 10 200 10 & 
srun --partition=core -c 12 --time=3-00:00:00 -J xgb8 python xgboost_model.py 8 binary:logistic 0

# Random Forest

## Inner fold - find HP

In [16]:
rf_df = pd.read_csv('results_v5_rf_ecfp/all-nested-cv-perf-scores.csv', header=0, sep="\t")
rf_df.head()

,sample,ofold,ifold,al,target,hp,auc,precision,recall,kappa
0,1,1,0,6,0,class_weight_balanced-max_features_auto-n_esti...,0.775111,0.763749,0.816901,0.545830
1,1,1,0,6,0,class_weight_balanced-max_features_auto-n_esti...,0.768052,0.758327,0.809760,0.531926
2,1,1,0,6,0,class_weight_balanced-max_features_auto-n_esti...,0.773379,0.769946,0.811429,0.543322
3,1,1,0,6,100,class_weight_balanced-max_features_auto-n_esti...,0.503381,0.006873,0.363636,0.013154
4,1,1,0,6,100,class_weight_balanced-max_features_auto-n_esti...,0.503381,0.006873,0.363636,0.013154


In [17]:
# separate inner and out loop results
rf_inner = rf_df.loc[rf_df['ifold'].isin([1,2])].copy(deep=True).drop(['al','sample'],axis=1)
rf_outer = rf_df.loc[rf_df['ifold'].isin([0])].copy(deep=True).drop(['al','sample'],axis=1)
del rf_df

In [18]:
# select best HP per fold
hp_selection = selectHP('auc',rf_inner)

In [19]:
hp_selection.shape

(526, 4)

## Outer folds - model evaluations

In [20]:
# use the selected HP to find out the performance in outer loop
perf_fold1, perf_fold2, perf_fold3 = getOuterLoopPerformance(rf_outer, hp_selection)

perf_fold1.to_csv("results_v5_rf_ecfp/perf_fold1.csv", sep="\t", index=None)
perf_fold2.to_csv("results_v5_rf_ecfp/perf_fold2.csv", sep="\t", index=None)
perf_fold3.to_csv("results_v5_rf_ecfp/perf_fold3.csv", sep="\t", index=None)

In [21]:
# hp selection based on AUC
print("           fold1           fold2           fold3")
print("AUC      =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['auc'].mean(),
                                                                           perf_fold1['auc'].std(),
                                                                           perf_fold2['auc'].mean(),
                                                                           perf_fold2['auc'].std(),
                                                                           perf_fold3['auc'].mean(),
                                                                           perf_fold3['auc'].std()))

print("PRECISION=({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['precision'].mean(),
                                                                           perf_fold1['precision'].std(),
                                                                           perf_fold2['precision'].mean(),
                                                                           perf_fold2['precision'].std(),
                                                                           perf_fold3['precision'].mean(),
                                                                           perf_fold3['precision'].std()))

print("RECALL   =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['recall'].mean(),
                                                                           perf_fold1['recall'].std(),
                                                                           perf_fold2['recall'].mean(),
                                                                           perf_fold2['recall'].std(),
                                                                           perf_fold3['recall'].mean(),
                                                                           perf_fold3['recall'].std()))

print("KAPPA    =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['kappa'].mean(),
                                                                           perf_fold1['kappa'].std(),
                                                                           perf_fold2['kappa'].mean(),
                                                                           perf_fold2['kappa'].std(),
                                                                           perf_fold3['kappa'].mean(),
                                                                           perf_fold3['kappa'].std()))

           fold1           fold2           fold3
AUC      =(0.6444+/-0.1137,0.6409+/-0.1123,0.6379+/-0.1123)
PRECISION=(0.4646+/-0.3641,0.4581+/-0.3664,0.4505+/-0.3637)
RECALL   =(0.7294+/-0.2732,0.7112+/-0.2736,0.7141+/-0.2791)
KAPPA    =(0.3270+/-0.2347,0.3185+/-0.2304,0.3151+/-0.2341)


# SVM

## Inner fold - find HP

In [5]:
svm_df = pd.read_csv('results_v5_svm_ecfp/all-nested-cv-perf-scores.tsv', header=0, sep="\t")
svm_df.head()

,sample,ofold,ifold,al,target,hp,auc,precision,recall,kappa
0,1,1,0,6,0,C_300-dual_True-loss_squared_hinge-penalty_l2,0.666746,0.620449,0.729508,0.327439
1,1,1,0,6,0,C_100-dual_True-loss_squared_hinge-penalty_l2,0.670803,0.618900,0.735727,0.335026
2,1,1,0,6,0,C_30-dual_True-loss_squared_hinge-penalty_l2,0.685857,0.692486,0.729201,0.369413
3,1,1,0,6,0,C_10-dual_True-loss_squared_hinge-penalty_l2,0.690788,0.695585,0.734260,0.379106
4,1,1,0,6,0,C_1-dual_True-loss_squared_hinge-penalty_l2,0.727089,0.761425,0.755573,0.454614


In [6]:
# separate inner and out loop results
svm_inner = svm_df.loc[svm_df['ifold'].isin([1,2])].copy(deep=True).drop(['al','sample'],axis=1)
svm_outer = svm_df.loc[svm_df['ifold'].isin([0])].copy(deep=True).drop(['al','sample'],axis=1)
del svm_df

In [7]:
# select best HP per fold
hp_selection = selectHP('auc',svm_inner)

## Outer fold - model evaluation

In [10]:
!pwd

/projects/cc/kxsw653/ExCAPE-ML_v5/IT4I-data/qsar_models


In [8]:
# use the selected HP to find out the performance in outer loop
perf_fold1, perf_fold2, perf_fold3 = getOuterLoopPerformance(svm_outer, hp_selection)

perf_fold1.to_csv("results_v5_svm_ecfp/perf_fold1.csv", sep="\t", index=None)
perf_fold2.to_csv("results_v5_svm_ecfp/perf_fold2.csv", sep="\t", index=None)
perf_fold3.to_csv("results_v5_svm_ecfp/perf_fold3.csv", sep="\t", index=None)

In [9]:
perf_fold1.sort_values

,ofold,ifold,target,hp,auc,precision,recall,kappa
0,1,0,0,C_0.1-dual_True-loss_hinge-penalty_l2,0.764828,0.815647,0.780578,0.532616
1,1,0,1,C_0.01-dual_True-loss_hinge-penalty_l2,0.795744,0.780059,0.801205,0.591688
2,1,0,2,C_100-dual_True-loss_hinge-penalty_l2,0.515563,0.033141,0.140244,0.050047
3,1,0,3,C_100-dual_False-loss_squared_hinge-penalty_l1,0.512383,0.025000,0.088235,0.038578
4,1,0,4,C_300-dual_True-loss_hinge-penalty_l2,0.504152,0.009724,0.071006,0.014484
5,1,0,5,C_100-dual_True-loss_hinge-penalty_l2,0.499914,0.000000,0.000000,-0.000312
6,1,0,6,C_300-dual_True-loss_squared_hinge-penalty_l2,0.514861,0.046445,0.148624,0.043847
7,1,0,7,C_300-dual_True-loss_squared_hinge-penalty_l2,0.550642,0.102848,0.383481,0.158863
8,1,0,8,C_300-dual_True-loss_squared_hinge-penalty_l2,0.504614,0.015766,0.119701,0.015632
9,1,0,9,C_300-dual_False-loss_squared_hinge-penalty_l1,0.500971,0.004515,0.039867,0.003423


In [26]:
# hp selection based on AUC
print("           fold1           fold2           fold3")
print("AUC      =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['auc'].mean(),
                                                                           perf_fold1['auc'].std(),
                                                                           perf_fold2['auc'].mean(),
                                                                           perf_fold2['auc'].std(),
                                                                           perf_fold3['auc'].mean(),
                                                                           perf_fold3['auc'].std()))

print("PRECISION=({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['precision'].mean(),
                                                                           perf_fold1['precision'].std(),
                                                                           perf_fold2['precision'].mean(),
                                                                           perf_fold2['precision'].std(),
                                                                           perf_fold3['precision'].mean(),
                                                                           perf_fold3['precision'].std()))

print("RECALL   =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['recall'].mean(),
                                                                           perf_fold1['recall'].std(),
                                                                           perf_fold2['recall'].mean(),
                                                                           perf_fold2['recall'].std(),
                                                                           perf_fold3['recall'].mean(),
                                                                           perf_fold3['recall'].std()))

print("KAPPA    =({:.4f}+/-{:.4f},{:.4f}+/-{:.4f},{:.4f}+/-{:.4f})".format(perf_fold1['kappa'].mean(),
                                                                           perf_fold1['kappa'].std(),
                                                                           perf_fold2['kappa'].mean(),
                                                                           perf_fold2['kappa'].std(),
                                                                           perf_fold3['kappa'].mean(),
                                                                           perf_fold3['kappa'].std()))

           fold1           fold2           fold3
AUC      =(0.6700+/-0.1168,0.6710+/-0.1128,0.6691+/-0.1138)
PRECISION=(0.5254+/-0.3149,0.5292+/-0.3114,0.5257+/-0.3107)
RECALL   =(0.5758+/-0.2999,0.5680+/-0.2979,0.5763+/-0.2901)
KAPPA    =(0.3430+/-0.2323,0.3454+/-0.2266,0.3482+/-0.2309)


# ExNET 

## Inner Loop - HP search

In [28]:
target_map = pd.read_csv('../../DATA/bioactivities/MAP_TARGETS_ExCAPE-ml.csv',header=None, sep=',', index_col=1, squeeze=True)
header=['sample','ofold','ifold','al','hp', 'step']
useless = [header.append(x) for x in target_map.values]
del useless

In [29]:
# one columns per target +     hyperparameters
#exnet_inner = pd.read_csv("/projects/cc/kxsw653/ExCAPE-ML_v5/IT4I-data/qsar_models/results_v5_exnet/exnet_inner_metrics_lite.tsv", header = 0, sep="\t")
exnet_inner = pd.read_csv("/projects/cc/kxsw653/ExCAPE-ML_v5/IT4I-data/qsar_models/results_v5_exnet/target_wise/exnet_inner_fullpack-target-wise.tsv", 
                          header = None, sep="\t", names=header)

In [30]:
exnet_inner.head()

,sample,ofold,ifold,al,hp,step,EGFR,ERBB2,ALDH1A1,APOBEC3G,...,SLC12A5,ADK,S1PR5,FBP1,C5AR1,THRA,F7,LTA4H,CMA1,HCAR2
0,1,1,1,6,1024_1024_1024-0_01-0_5-0_8-0_0,130000,0.639472,0.363534,0.519393,0.608180,...,0.385377,0.166154,0.211647,0.577778,0.398931,0.493761,0.562999,0.392628,0.447002,0.462589
1,1,1,1,6,1024_1024_1024-0_01-0_5-0_8-0_0,170000,0.649232,0.388696,0.522374,0.627334,...,0.380154,0.138462,0.192369,0.573333,0.435860,0.677514,0.369463,0.370994,0.449206,0.465514
2,1,1,1,6,1024_1024_1024-0_01-0_5-0_8-0_0,210000,0.658029,0.399156,0.521756,0.623755,...,0.370533,0.126154,0.169076,0.580000,0.430029,0.738194,0.280501,0.358173,0.438536,0.482181
3,1,1,1,6,1024_1024_1024-0_01-0_5-0_8-0_0,260000,0.652803,0.535138,0.520949,0.621915,...,0.390874,0.123077,0.167470,0.584444,0.417396,0.780034,0.322077,0.346154,0.443122,0.555496
4,1,1,1,6,1024_1024_1024-0_01-0_5-0_8-0_0,300000,0.654717,0.707544,0.523126,0.611750,...,0.411765,0.121538,0.153815,0.571111,0.375121,0.794715,0.345861,0.331731,0.436067,0.670745


In [31]:
exnet_inner_final = exnet_inner.loc[exnet_inner['step']==480000].drop(['sample', 'al'],axis=1)
exnet_inner_final.head()

,ofold,ifold,hp,step,EGFR,ERBB2,ALDH1A1,APOBEC3G,ATAD5,BRCA1,...,SLC12A5,ADK,S1PR5,FBP1,C5AR1,THRA,F7,LTA4H,CMA1,HCAR2
10,1,1,1024_1024_1024-0_01-0_5-0_8-0_0,480000,0.646505,0.723674,0.538322,0.626419,0.595862,0.652656,...,0.450797,0.133846,0.129719,0.584444,0.495141,0.801321,0.338235,0.323718,0.453263,0.660461
25,1,1,1024_1024_1024-0_01-0_5-0_8-0_4,480000,0.655590,0.720136,0.555043,0.644933,0.602999,0.672664,...,0.493678,0.141538,0.162651,0.544444,0.568513,0.796428,0.323529,0.312500,0.511199,0.633333
40,1,1,1024_1024_1024-0_01-0_5-1_0-0_0,480000,0.659623,0.734540,0.559255,0.662940,0.604420,0.670085,...,0.510720,0.176923,0.851004,0.542222,0.664237,0.802055,0.344953,0.296474,0.523192,0.585018
55,1,1,1024_1024_1024-0_01-0_5-1_0-0_4,480000,0.667653,0.735606,0.566943,0.664524,0.606412,0.678587,...,0.413964,0.244615,0.886345,0.600000,0.692420,0.805725,0.377905,0.310096,0.485891,0.620656
70,1,1,1024_1024_1024-0_1-0_5-0_8-0_0,480000,0.684571,0.748714,0.560915,0.755989,0.653678,0.845998,...,0.411215,0.798462,0.895984,0.566667,0.752187,0.770981,0.464415,0.710737,0.492945,0.553546


In [32]:
# get the mean across inner folds for each HP
mean_perf_per_ifold = exnet_inner_final.groupby(['ofold','hp']).mean().reset_index()

# get the mean across targets
perf = mean_perf_per_ifold.drop(['step','ifold'],axis=1).set_index(['ofold','hp']).T

# only the first one for each fold should be selected
print("fold-1:",perf[1].mean().sort_values(ascending=False).head(2))
print("fold-2:",perf[2].mean().sort_values(ascending=False).head(2))
print("fold-3:",perf[3].mean().sort_values(ascending=False).head(2))

fold-1: hp
2048_2048-0_1-0_5-1_0-0_4    0.759769
2048_2048-0_1-0_5-0_8-0_4    0.754120
dtype: float64
fold-2: hp
2048_2048-0_1-0_5-1_0-0_4    0.764706
2048_2048-0_1-0_5-0_8-0_4    0.756711
dtype: float64
fold-3: hp
2048_2048-0_1-0_5-1_0-0_4    0.757835
2048_2048-0_1-0_5-0_8-0_4    0.749826
dtype: float64


## Outer Loop - Model evaluation

In [69]:
# there is one columns per target
ex_outer = pd.read_csv('results_v5_exnet/target_wise/exnet_outer_fullpack-target-wise.tsv', header=None, sep="\t", names=header)
                      #dtype={'ofold':'category', 
                      #       'ifold':'category', 
                      #       'network':'category', 
                      #       'param':'category'})

# select the final checkpoints for each model, keep only checkpoints saved at same time in each outer loop
ex_outer_final_steps = ex_outer.loc[ex_outer['step']==1940000]

# get performance of selected HP from inner fold
selected_hp = '2048_2048-0_1-0_5-1_0-0_4'
perf = ex_outer_final_steps.loc[ex_outer_final_steps['hp']==selected_hp].drop(['sample','al','step','hp', 'ifold'],axis=1).set_index('ofold').T
perf.head()

ofold,1,2,3
EGFR,0.836050,0.872380,0.832454
ERBB2,0.900838,0.917334,0.924384
ALDH1A1,0.710809,0.695130,0.687927
APOBEC3G,0.857099,0.853981,0.804838
ATAD5,0.689444,0.707411,0.680767


In [66]:
ex_outer_final_steps.shape

(96, 532)

In [64]:
ex_outer_final_steps.loc[ex_outer_final_steps[4]==selected_hp]#.drop(['sample','al','step','hp', 'ifold'],axis=1)#.set_index('ofold')

,0,1,2,3,4,5,6,7,8,9,...,522,523,524,525,526,527,528,529,530,531
3030,1,1,0,6,2048_2048-0_1-0_5-1_0-0_4,1940000,0.836050,0.900838,0.710809,0.857099,...,0.518743,0.669519,0.951564,0.690326,0.850742,0.869703,0.676733,0.799682,0.787284,0.795324
9360,1,2,0,6,2048_2048-0_1-0_5-1_0-0_4,1940000,0.872380,0.917334,0.695130,0.853981,...,0.530819,0.884633,0.891255,0.713287,0.702344,0.884457,0.674712,0.827938,0.726667,0.822149
15822,1,3,0,6,2048_2048-0_1-0_5-1_0-0_4,1940000,0.832454,0.924384,0.687927,0.804838,...,0.669599,0.890769,0.881526,0.931111,0.788630,0.804991,0.673566,0.893429,0.552822,0.683954


In [34]:
perf.to_csv("results_v5_exnet/auc_perf_ofolds.csv", sep="\t")

In [ ]:
# get confusion matrix based metrics

In [70]:
fold1 = pd.read_csv("results_v5_exnet/nested-cv-predictions/predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4.testmetrics", header=0, sep="\t")
fold1.head(7)

,filename,target,auc,recall,precision,f1,cohen_kappa
0,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,EGFR,0.768526,0.886135,0.665503,0.760133,0.412870
1,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,ERBB2,0.650448,0.727273,0.589074,0.650919,0.232286
2,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,ALDH1A1,0.672300,0.080692,0.129032,0.099291,0.092433
3,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,APOBEC3G,0.703815,0.016667,0.013333,0.014815,0.013825
4,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,ATAD5,0.638408,0.081848,0.096743,0.088674,0.079353
5,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,BRCA1,0.869893,0.084821,0.022196,0.035185,0.032392
6,predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4,EHMT2,0.584642,0.176606,0.100065,0.127748,0.046793


In [ ]:
# Why the f*** is AUC different in fold1 and in perf ??????????

# SMURFF -- WRONG look and smruff HP selection.ipynb

## Inner folds - HP selection

In [43]:
smurff_inner = pd.read_csv('results_v5_smurff/inner_folds_final.csv.gz', usecols=[1,8,12,16,20,24,44,45,46,47,48,49], compression='gzip', header=0, sep='\t')# ,error_bad_lines=False
smurff_inner = smurff_inner.dropna(axis=0, how="all")

In [44]:
# check the number of inner fold records

smurff_inner['cv_id'].value_counts()
# --> there are some missing data ... 

s1_c2_i1    327172
s1_c3_i2    324542
s1_c3_i1    323490
s1_c1_i1    322964
s1_c1_i2    322964
s1_c2_i2    280884
Name: cv_id, dtype: int64

In [45]:
# format the data in a way that works: create one HP columns with all hyperparam
smurff_inner['ofold'] = smurff_inner['cv_id'].str.split('_',expand=True)[1]
smurff_inner['ifold'] = smurff_inner['cv_id'].str.split('_',expand=True)[2]
smurff_inner['hp'] = smurff_inner['samples'].astype(str).str.cat(smurff_inner['nl'].astype(str), sep="_nl").str.cat(smurff_inner['bp'].astype(str), sep="_bp").str.cat(smurff_inner['tp'].astype(str), sep="_tp")
smurff_inner = smurff_inner.drop(['cv_id','nl','bp','tp'],axis=1)

In [46]:
smurff_inner.head()

,col_id,auc_l6,prec_l6,rec_l6,f1_l6,kappa_l6,burnins,samples,ofold,ifold,hp
0,0,0.754203,0.585284,0.813953,0.680934,0.115081,200,1000,c1,i1,1000_nl16_bp10_tp10
1,1,0.819002,0.592697,0.864754,0.703333,0.260595,200,1000,c1,i1,1000_nl16_bp10_tp10
2,2,0.661777,0.148936,0.010574,0.019746,0.029248,200,1000,c1,i1,1000_nl16_bp10_tp10
3,3,0.550285,0.000000,0.000000,0.000000,0.000510,200,1000,c1,i1,1000_nl16_bp10_tp10
4,4,0.629571,0.076087,0.005121,0.009596,0.046246,200,1000,c1,i1,1000_nl16_bp10_tp10


In [47]:
# average inner fold results over the six inner fold runs
smurff_inner.groupby(['hp']).mean().reset_index().sort_values('auc_l6', ascending=False)

,hp,col_id,auc_l6,prec_l6,rec_l6,f1_l6,kappa_l6,burnins,samples
342,1800_nl64_bp10_tp1,262.5,0.760919,0.475949,0.453340,0.426336,0.125686,200.0,1800.0
129,1200_nl64_bp1_tp1,262.5,0.758250,0.470010,0.443106,0.415792,0.112477,200.0,1200.0
165,1300_nl64_bp1_tp1,262.5,0.758203,0.469519,0.442859,0.415460,0.112217,200.0,1300.0
93,1100_nl64_bp1_tp1,262.5,0.758076,0.469749,0.442975,0.415863,0.111988,200.0,1100.0
21,1000_nl64_bp1_tp1,262.5,0.757817,0.470004,0.442700,0.415685,0.111682,200.0,1000.0
201,1400_nl64_bp1_tp1,262.5,0.757741,0.467570,0.441540,0.411996,0.108204,200.0,1400.0
237,1500_nl64_bp1_tp1,262.5,0.757560,0.467026,0.441219,0.411652,0.107899,200.0,1500.0
631,900_nl64_bp1_tp1,262.5,0.757338,0.470834,0.442255,0.415361,0.110624,200.0,900.0
273,1600_nl64_bp1_tp1,262.5,0.757233,0.467038,0.441305,0.411375,0.107717,200.0,1600.0
198,1400_nl64_bp10_tp1,262.5,0.756999,0.462895,0.445471,0.415012,0.113179,200.0,1400.0


In [48]:
# find out the best HPs for each fold by averaging over the two inner folds in each fold 
# no restrinction made on the number of samples
mean=smurff_inner.groupby(['hp','ofold']).mean().reset_index().sort_values('auc_l6', ascending=False)
mean.groupby(['ofold']).first().reset_index()

,ofold,hp,col_id,auc_l6,prec_l6,rec_l6,f1_l6,kappa_l6,burnins,samples
0,c1,1200_nl64_bp10_tp1,262.5,0.757327,0.469492,0.442140,0.413736,0.108458,200.0,1200.0
1,c2,1500_nl64_bp1_tp1,262.5,0.759611,0.464144,0.448272,0.411971,0.108071,200.0,1500.0
2,c3,1800_nl64_bp10_tp1,262.5,0.760919,0.475949,0.453340,0.426336,0.125686,200.0,1800.0


In [ ]:
# => fold 1:  samples = 1200, nl=64, bp=10, tp=1
# => fold 2:  samples = 1500, nl=64, bp=1 , tp=1
# => fold 3:  samples = 1800, nl=64, bp=10, tp=1

In [143]:
# alternative 
# Select the best HPs at sample=1700
smurff_inner1700 = smurff_inner.loc[smurff_inner['samples']==1700].copy(deep=True)
smurff_inner1700.groupby(['hp','ofold']).mean().reset_index().sort_values('auc_l6', ascending=False).groupby(['ofold']).first().reset_index()

,ofold,hp,col_id,auc_l6,prec_l6,rec_l6,f1_l6,kappa_l6,burnins,samples
0,c1,1700_nl64_bp10_tp1,262.5,0.756770,0.461765,0.442699,0.412033,0.106115,200.0,1700.0
1,c2,1700_nl64_bp1_tp1,262.5,0.758963,0.464874,0.448420,0.411268,0.107306,200.0,1700.0
2,c3,1700_nl64_bp10_tp1,262.5,0.756491,0.456837,0.448964,0.415776,0.116478,200.0,1700.0


In [ ]:
smurff_inner1700['ofold'].value_counts()

## Outer folds - Model evaluations

In [4]:
smurff_outer = pd.read_csv('results_v5_smurff/outer_folds_final.csv.gz', usecols=[1,8,12,16,20,24,44,45,46,47,48,49], compression='gzip', header=0, sep='\t')# ,error_bad_lines=False
smurff_outer = smurff_outer.dropna(axis=0, how='all')

In [5]:
smurff_outer.head()

,col_id,auc_l6,prec_l6,rec_l6,f1_l6,kappa_l6,cv_id,burnins,samples,nl,bp,tp
0,0,0.803692,0.715830,0.872192,0.786313,0.125994,s1_c1,200,1000,16,10,10
1,1,0.851680,0.715618,0.900293,0.797403,0.099906,s1_c1,200,1000,16,10,10
2,2,0.652197,0.022222,0.001441,0.002706,0.022962,s1_c1,200,1000,16,10,10
3,3,0.608130,0.000000,0.000000,0.000000,0.000817,s1_c1,200,1000,16,10,10
4,4,0.638063,0.061856,0.004862,0.009016,0.039437,s1_c1,200,1000,16,10,10


In [50]:
# count the number of records per fold
smurff_outer['cv_id'].value_counts()

# missing data as well

s1_c3    321912
s1_c2    321912
s1_c1    303502
Name: cv_id, dtype: int64

In [51]:
# format the data in a way that works: create one HP columns with all hyperparam
smurff_outer['ofold'] = smurff_outer['cv_id'].str.split('_',expand=True)[1]
smurff_outer['hp'] = smurff_outer['samples'].astype(str).str.cat(smurff_outer['nl'].astype(str), sep="_nl").str.cat(smurff_outer['bp'].astype(str), sep="_bp").str.cat(smurff_outer['tp'].astype(str), sep="_tp")
smurff_outer = smurff_outer.drop(['cv_id','nl','bp','tp'],axis=1)

In [144]:
hp_c1 = "1700_nl64_bp10_tp1"
hp_c2 = "1700_nl64_bp1_tp1"
hp_c3 = "1700_nl64_bp10_tp1"


smurff_outer_c1 = smurff_outer.loc[(smurff_outer['ofold']=='c1')&(smurff_outer['hp']==hp_c1)]
smurff_outer_c2 = smurff_outer.loc[(smurff_outer['ofold']=='c2')&(smurff_outer['hp']==hp_c2)]
smurff_outer_c3 = smurff_outer.loc[(smurff_outer['ofold']=='c3')&(smurff_outer['hp']==hp_c3)]


In [145]:
smurff_outer_c3.head()

,col_id,auc_l6,prec_l6,rec_l6,f1_l6,kappa_l6,burnins,samples,ofold,hp
799520,0,0.810862,0.685915,0.755039,0.718819,0.300324,200,1700,c3,1700_nl64_bp10_tp1
799521,1,0.911781,0.685015,0.918033,0.784588,0.257552,200,1700,c3,1700_nl64_bp10_tp1
799522,2,0.683782,0.000000,0.000000,0.000000,0.004332,200,1700,c3,1700_nl64_bp10_tp1
799523,3,0.803055,0.000000,0.000000,0.000000,-0.000015,200,1700,c3,1700_nl64_bp10_tp1
799524,4,0.657538,0.250000,0.001463,0.002909,0.011847,200,1700,c3,1700_nl64_bp10_tp1


In [152]:
smurff_outer_c1.to_csv('results_v5_smurff/perf_ofold1.csv', sep="\t", index=None)
smurff_outer_c2.to_csv('results_v5_smurff/perf_ofold2.csv', sep="\t", index=None)
smurff_outer_c3.to_csv('results_v5_smurff/perf_ofold3.csv', sep="\t", index=None)

In [147]:
print("auc:{} +/-{}".format(smurff_outer_c1['auc_l6'].mean(), smurff_outer_c1['auc_l6'].std()))
print("auc:{} +/-{}".format(smurff_outer_c2['auc_l6'].mean(), smurff_outer_c2['auc_l6'].std()))
print("auc:{} +/-{}".format(smurff_outer_c3['auc_l6'].mean(), smurff_outer_c3['auc_l6'].std()))

auc:0.7838133771642667 +/-0.12118727211881054
auc:0.7793024205155963 +/-0.12706133717521323
auc:0.7833483080113223 +/-0.12398470563634535


# Stich everything together

In [41]:
xgb1 = pd.read_csv('results_v5_xgboost_ecfp/perf_fold1.csv', header=0, sep="\t")
xgb2 = pd.read_csv('results_v5_xgboost_ecfp/perf_fold2.csv', header=0, sep="\t")
xgb3 = pd.read_csv('results_v5_xgboost_ecfp/perf_fold3.csv', header=0, sep="\t")

x1 = xgb1.drop(['ofold','ifold','hp'],axis=1)
x2 = xgb2.drop(['ofold','ifold','hp'],axis=1)
x3 = xgb3.drop(['ofold','ifold','hp'],axis=1)
x1.columns = ['target','1_xgb_auc', '1_xgb_recall', '1_xgb_precision', '1_xgb_kappa']
x2.columns = ['target','2_xgb_auc', '2_xgb_recall', '2_xgb_precision', '2_xgb_kappa']
x3.columns = ['target','3_xgb_auc', '3_xgb_recall', '3_xgb_precision', '3_xgb_kappa']

all_xgb = x1.merge(x2, on='target').merge(x3, on='target')

In [42]:
svm1 = pd.read_csv('results_v5_svm_ecfp/perf_fold1.csv', header=0, sep="\t")
svm2 = pd.read_csv('results_v5_svm_ecfp/perf_fold2.csv', header=0, sep="\t")
svm3 = pd.read_csv('results_v5_svm_ecfp/perf_fold3.csv', header=0, sep="\t")

s1 = svm1.drop(['ofold','ifold','hp'],axis=1)
s2 = svm2.drop(['ofold','ifold','hp'],axis=1)
s3 = svm3.drop(['ofold','ifold','hp'],axis=1)
s1.columns = ['target','1_svm_auc', '1_svm_precision', '1_svm_recall',  '1_svm_kappa']
s2.columns = ['target','2_svm_auc', '2_svm_precision', '2_svm_recall',  '2_svm_kappa']
s3.columns = ['target','3_svm_auc', '3_svm_precision', '3_svm_recall',  '3_svm_kappa']

all_svm = s1.merge(s2, on='target').merge(s3, on='target')

In [43]:
rf1 = pd.read_csv('results_v5_rf_ecfp/perf_fold1.csv', header=0, sep="\t")
rf2 = pd.read_csv('results_v5_rf_ecfp/perf_fold2.csv', header=0, sep="\t")
rf3 = pd.read_csv('results_v5_rf_ecfp/perf_fold3.csv', header=0, sep="\t")

r1 = rf1.drop(['ofold','ifold','hp'],axis=1)
r2 = rf2.drop(['ofold','ifold','hp'],axis=1)
r3 = rf3.drop(['ofold','ifold','hp'],axis=1)

r1.columns = ['target','1_rf_auc', '1_rf_precision', '1_rf_recall',  '1_rf_kappa']
r2.columns = ['target','2_rf_auc', '2_rf_precision', '2_rf_recall',  '2_rf_kappa']
r3.columns = ['target','3_rf_auc', '3_rf_precision', '3_rf_recall',  '3_rf_kappa']

all_rf = r1.merge(r2, on='target').merge(r3, on='target')

In [97]:
all_rf.head()

,target,1_auc,1_precision,1_recall,1_kappa,2_auc,2_precision,2_recall,2_kappa,3_auc,3_precision,3_recall,3_kappa
0,0,0.773379,0.769946,0.811429,0.543322,0.813441,0.769542,0.847181,0.626088,0.774280,0.727132,0.757674,0.551139
1,1,0.794876,0.712610,0.849650,0.590978,0.883388,0.888660,0.907368,0.764070,0.877033,0.823770,0.885463,0.763261
2,2,0.505728,0.011527,0.615385,0.022278,0.507672,0.015385,0.785714,0.029822,0.506035,0.012085,0.888889,0.023603
3,3,0.504159,0.008333,0.333333,0.016217,0.499996,0.000000,0.000000,-0.000014,0.499985,0.000000,0.000000,-0.000058
4,4,0.501981,0.004052,0.333333,0.007743,0.505276,0.010638,0.600000,0.020492,0.503969,0.008047,0.478261,0.015430


In [159]:
prk1 = pd.read_csv("results_v5_exnet/nested-cv-predictions/", header=0, sep="\t")
prk1['cohen_kappa'].mean()

0.21552019945540535

In [133]:
exnet = pd.read_csv("results_v5_exnet/auc_perf_ofolds.csv", header=0, sep="\t", index_col=0).reset_index()
exnet.columns = ['target', '1_exnet_auc', '2_exnet_auc', '3_exnet_auc']

prk1 = pd.read_csv("results_v5_exnet/nested-cv-predictions/predictions-1-1-0-6-2048_2048-0_1-0_5-1_0-0_4.testmetrics", header=0, sep="\t")
prk2 = pd.read_csv("results_v5_exnet/nested-cv-predictions/predictions-1-2-0-6-2048_2048-0_1-0_5-1_0-0_4.testmetrics", header=0, sep="\t")
prk3 = pd.read_csv("results_v5_exnet/nested-cv-predictions/predictions-1-3-0-6-2048_2048-0_1-0_5-1_0-0_4.testmetrics", header=0, sep="\t")

prk1 = prk1.drop(['filename','auc'],axis=1)
prk2 = prk2.drop(['filename','auc'],axis=1)
prk3 = prk3.drop(['filename','auc'],axis=1)

prk1.columns=['target','1_exnet_recall', '1_exnet_precision', '1_exnet_f1', '1_exnet_kappa']
prk2.columns=['target','2_exnet_recall', '2_exnet_precision', '2_exnet_f1', '2_exnet_kappa']
prk3.columns=['target','3_exnet_recall', '3_exnet_precision', '3_exnet_f1', '3_exnet_kappa']

all_exnet = exnet.merge(prk1, on='target').merge(prk2, on='target').merge(prk3, on='target')

In [156]:
smu1 = pd.read_csv("results_v5_smurff/nested-cv-evaluations/perf_smurff_ofold1.csv", header=0, sep="\t")
smu2 = pd.read_csv("results_v5_smurff/nested-cv-evaluations/perf_smurff_ofold2.csv", header=0, sep="\t")
smu3 = pd.read_csv("results_v5_smurff/nested-cv-evaluations/perf_smurff_ofold3.csv", header=0, sep="\t")

#smu1 = smu1.drop(['burnins', 'samples', 'ofold', 'hp'],axis=1)
#smu2 = smu2.drop(['burnins', 'samples', 'ofold', 'hp'],axis=1)
#smu3 = smu3.drop(['burnins', 'samples', 'ofold', 'hp'],axis=1)
#
#
#smu1.columns=['target', '1_smurff_auc', '1_smurff_precision', '1_smurff_recall', '1_smurff_f1', '1_smurff_kappa']
#smu2.columns=['target', '2_smurff_auc', '2_smurff_precision', '2_smurff_recall', '2_smurff_f1', '2_smurff_kappa']
#smu3.columns=['target', '3_smurff_auc', '3_smurff_precision', '3_smurff_recall', '3_smurff_f1', '3_smurff_kappa']
#
#all_smurff = smu1.merge(smu2, on='target').merge(smu3, on='target')


In [157]:
all_smurff.columns

Index(['target', '1_smurff_auc', '1_smurff_precision', '1_smurff_recall',
       '1_smurff_f1', '1_smurff_kappa', '2_smurff_auc', '2_smurff_precision',
       '2_smurff_recall', '2_smurff_f1', '2_smurff_kappa', '3_smurff_auc',
       '3_smurff_precision', '3_smurff_recall', '3_smurff_f1',
       '3_smurff_kappa'],
      dtype='object')

In [154]:
all_perf = all_exnet.merge(all_xgb, left_index=True, right_on='target').merge(all_smurff, on='target').merge(all_svm, on='target').merge(all_rf, on='target')

In [155]:
all_perf.to_csv('ALL_PERF.csv', sep="\t")

In [44]:
all_perf.head()

NameError: name 'all_perf' is not defined